# Databases and SQL

In this notebook, we will learn about databases and introduce structured query language (SQL).  SQL is the main way one interacts with relational databases.  SQL is much different from traditional programming languages.  While SQL syntax initially seems straightforward, it can quickly become confusing as you try more complicated queries - a single SQL statement can contain the complexity of an entire Python or Java program.

We will learn about the concept of a database, the basics of SQL, then use it to provide a pattern for exploring the class data sets.

## Learning objectives

- Become familiar with the basic syntax, structure, and uses of SQL.
- Get hands on experience writing and running SQL queries.
- Learn and use descriptive SQL queries to familiarize yourself with the class data sets.

## What is a Database?

Before we dive into the specifics of SQL, let's quickly discuss the basics of databases and relational databases in particular.

- **database**: a database is a collection of data about entities. It can be more or less structured, depending on the type of database, and can include information about the relationships between entities.
- **database management system (DBMS)**: DBMS is a system that provides infrastructure for storing, managing and interacting with databases. It generally includes 3 elements: how to store data, a query language, support for transactions and crash recovery.

More specifically, in a **_relational database management system (RDBMS)_**, or relational database, data records are stored in **_tables_**, each of which:

- has a predefined set of **_columns_** - the pieces of information captured for each record in a table.
- stores data records as **_rows_** in the table, where each row has a place to store a value for every column in the table.

Tables, including their columns, column types and relationships with other tables, are defined in a database **_schema_**.

When tables contain a **_primary key_**, one or more columns that uniquely identify each row in a given table, rows in one table can also be explicitly related to rows in other tables through **_foreign key_** columns that hold the primary key of the related row.

The data for this class is stored using the Postgresql RDBMS.

## The data
In this notebook, we will explore the class data obtained from Federal Explorer

### ada_pub - `projects`
This table contains information on all grants, such as Title, PI, Costs, Institution, Funding Agency, Start and End Dates and Location information:
* project_id                    
* project_terms                 
* project_title                 
* department                    
* agency                        
* ic_center                        
* project_number                
* project_start_date            
* project_end_date              
* contact_pi_project_leader     
* other_pis                     
* congressional_district        
* duns_number                   
* organization_name             
* organization_city             
* organization_state            
* organization_zip              
* organization_country          
* budget_start_date             
* budget_end_date               
* cfda_code                     
* fy                            
* fy_total_cost                 
* fy_total_cost_sub_projects

### ada_pub - `abstracts`
This table includes the project number and the abstract of the corresponding grant submission:
* project_id    
* abstract      

### ada_pub - `publications`
We have two publication tables: publications_hhs and publications_other. HHS publication come from the NIH Reporter whereas the other publications come from Federal Reporter. The content of these files differs a bit. However both contain publication titles and author names and the grant number the publication is linked with.

HHS table: 
* affiliation         
* author_list           
* country             
* issn                  
* journal_issue          
* journal_title         
* journal_title_abbr    
* journal_volume         
* lang                 
* page_number           
* pmc_id               
* pmid                 
* pub_date             
* pub_title             
* pub_year            
* project_number     

OTHER table:
* project_number    
* title             
* authors_list

# Setup
To connect with the database we will be using Pythons Pandas package. There are other ways you can connect to the database as well. There are graphical interfaces (PGAdmin), the command line (psql) and other python packages (psycopg2, SQlAlchemy).

## Pandas
We'll use the [pandas package](http://pandas.pydata.org/) to populate `pandas` DataFrames from the results of SQL queries.  `pandas` uses a `sqlalchemy` database engine to connect to databases and run queries.  It then reads data returned from a given SQL query and further processes it to store it in a tabular data format called a "DataFrame" (a term that will be familiar for those with R or STATA experience).

DataFrames allow for easy statistical analysis, and can be directly used for machine learning.  They also load your entire result set into memory by default, and so are not suitable for really large data sets.

In the code cell below, we'll use `sqlalchemy` to connect to the database, then we'll give this engine to pandas and let it retrieve and process data.

In [ ]:
# create postgresql connection - three '/' indicate to use default host, port, username, and password
from sqlalchemy import create_engine
engine = create_engine('postgresql:///ada_pub')

In [ ]:
# The next step is to define the query
QUERY = '''
THIS IS WHERE YOUR QUERY GOES ;
'''

In [ ]:
# Save the output in a datframe
example=pd.read_sql_query(QUERY,con=engine)

# SQL basics
SQL is a quirky language, designed for a very specific purpose: to interact with relational data. It isn't structured like other languages, and while it can make data access easy, it also can make tasks that would be easy in other languages (though perhaps not exceptionally performant) confoundingly complex.  


## `SELECT` - Querying the database
The basic method of querying the database is to use a select statement. To retrieve all columns and rows in the hh_indcase_spells table, we can use the following statement:

    SELECT *
    FROM projects;

where:

- _Columns_ or _variables_ that would like returned are put in the **SELECT clause** (after the word "SELECT" but before the word "FROM").  An asterisk ( "\*" ) is a wildcard - it will return all columns for a given table.
- For each table you reference, put the name of the _schema_ in which the table lives, a period, and then the _table_ after the word "FROM" in the **FROM clause**.

    - Example: "`FROM projects`":

        - "projects" is the table name.

- It is considered good style to capitalize the parts of an SQL query that are part of the SQL language (SELECT, FROM, WHERE, etc.), and not variables, table names, or values you are filtering on or searching for.
- Although it isn't always necessary in PostgreSQL, you should end SQL statements with a semi-colon.  It isn't required everywhere, but it is required in some contexts so better to be aware and get into the habit.
- white spaces and line breaks has no effect on the outcome of the query. It is recommended to use line breaks to improve readability.

### LIMIT clause
It is often useful to limit the number of _rows_ retrived during data exploration because as a database grows toward "big data", retrieving all rows in a table can take a long time, and storing them all in a program that lets you view the results can take a lot of memory. To retrieve the first 1000 rows of the `project` table, a **LIMIT clause** is added to the end of a query:

    SELECT *
    FROM projects
    LIMIT 1000;

### SELECT specific columns
Often times only certain _columns_ are needed for a specific task. Instead of specifying “all” columns using the "\*" , you can specify which columns you want by name, in a comma-delimited list after "SELECT". 

    SELECT project_number, project_start_date, project_end_date, agency
    FROM projects
    LIMIT 1000;

## WHERE keyword: subsetting the data

When exploring data for the first time, especially relatively large tables, it helps to first focus in on a relatively small subset while getting your bearings.  The SQL **`WHERE clause`** lets you filter queries.  To learn about `WHERE` clauses, lets explore a more focused question about our data:

First, we would like to make sure we are looking at the data concerning only 2014.

    SELECT *
    FROM projects
    WHERE fy = '2014'
    LIMIT 1000;

The WHERE keyword allows us to create one or more comparisons known as conditions that filter rows returned by a query. The simplest condition consists of a column name, a comparator, and then the column or value being compared to.  For this query we have only one condition: the value in the fy column should be 2014:

In SQL we use the `AND` and `OR` keywords to combine conditions.  `AND` requires two conditions or sets of conditions to evaluate to TRUE.  `OR` only requires one of a set of conditions to evaluate to TRUE.  To require start date to be both after "2015-01-01" `AND` end date to be before "2015-12-31":

    SELECT *
    FROM projects
    WHERE ( fy = '2014' ) AND ( agency = 'USDA' )
    LIMIT 1000;

_Note:_

- common comparison operators:

    - "**_`=`_**" - equal to
    - "**_`!=`_**" or "**_`<>`_**" - not equal to
    - "**_`<`_**" - less than
    - "**_`<=`_**" - less-than-or-equal-to
    - "**_`>`_**" - greater than
    - "**_`>=`_**" - greater-than-or-equal-to
    - "**_`LIKE`_**" and "**_`NOT LIKE`_**" - wild-card matching operator, where percent matches 0 or more characters ( "%" ) and an underscore matches any 1 character ( "_" ). Can only apply to string data.
    - "**_`IN( value_list )`_**" and "**_`NOT IN( value_list )`_**" - checks whether the value to the left of the "IN", usually a column's value in a given row, is either IN or NOT IN the list on the right of the IN.
    - "**_`IS NULL`_** and "**_`IS NOT NULL`_**" - The signifier of a row in a column not having a value is a special keyword: `NULL`.  To check for `NULL`, you use "`IS NULL`" or "`IS NOT NULL`", rather than "=" or "!=".

- we can wrap any comparison in a `WHERE` clause with a pair of parentheses to help improve code readability and explicitly specify the order of operations within the `WHERE` clause (especially helpful when you have complex filter strings with lots of `AND`s and `OR`s).

### `LIKE` comparison operator

The `LIKE` operator is a particularly useful tool and deserves an example (select the project title value starting with 'TECHNOLOGY'):

    SELECT *
    FROM projects
    WHERE project_title LIKE 'TECHNOLOGY%'
    LIMIT 1000;
    
### `NULL` - finding missing values

An example of looking for rows in which a particular column is `NULL`:

    /* find missing values */
    SELECT *
    FROM projects
    WHERE other_pis IS NULL
    LIMIT 1000;
    
### Types of data and `WHERE`

Basic data within a relational database is usually one of 2 broad types: _numeric_ and _text_, each of which has a slightly different syntax from the other when you place literal values in an SQL statement (like the 'food%' pattern above):

- when you place a text value directly into a query, you must enclose it in single-quotes.  Double-quotes have an entirely different meaning than single quotes in SQL, and can cause your query to either fail outright or return unexpected results.
- when you place a numeric value (integer or decimal) into a query, you must NOT enclose it in quotes.  You just put the numeric value directly in the query.

In addition, some operators only work with one of these two types of data or the other.  `LIKE`, for example, can only be used with text columns and values.

### `COUNT` and `DISTINCT` 

What to do if we want to find out how many grants are reported in 2010. We can do this using the `COUNT` clause. And we can assign a name to the result of our calculation using the `AS` statement. Put the keyword `AS` right after the column name or computation in the `SELECT` clause that we want to rename, followed by the new column name. You can also just put an asterisk inside `COUNT( * )` to count the number of rows that match your `WHERE` clause filter criteria.  To just count spells in 2010:

     SELECT COUNT( * ) AS grant_count
     FROM projects
     WHERE ( fy = 2010 );

Now we turn our attention to actually getting a count of DISTINCT agencies.  We only want to count a given agency once, even if they granted more than one grant within a year.  To do this, we use the **`DISTINCT`** keyword in our `SELECT` clause to only include each unique value in a given column once in query results, and we use agency as the way to identify the same agency when there are multiple spells:

    SELECT DISTINCT( agency )
    FROM projects
    WHERE ( fy = 2010 );
    
When you use `DISTINCT`, you place it in the `SELECT` clause, followed by one or more columns whose unique sets of values you want returned by the query.  `SELECT DISTINCT` will only return any given unique value or set of values once within a query.

Next we use the `COUNT` aggregate function to count the distinct agencies we've found:

    SELECT COUNT( DISTINCT( agency ) ) AS agency_count
    FROM projects
    WHERE ( fy = 2010 );

## Aside - Building a query bit by bit

- Back to [Table of Contents](#Table-of-Contents)

The most reliable and efficient way to implement an SQL query (or any programming project) of even modest complexity is to break up the work you need to do into small pieces and implement and test each unit of work, one-by-one, as we did in the examples abov..  In SQL, this is particularly relevant since a single SQL query can become very complex very quickly, and you want to understand exactly what is going on if you are planning on using the query to `CREATE` or `UPDATE` the database.  It can be tempting to just write out a giant query all at once then test, but this is often much more difficult to debug if you have problems.  Even once you become more experienced with SQL (and programming in general), it is still a good idea to built things piece by piece, testing each bit as you implement it.

## `GROUP BY` - Clustering columns based on column values

Now we turn to clustering rows based on their shared values in a column or columns of interest to examine the different agencies behind the grants.

The GROUP BY keyward lets you build a query that includes aggregate math functions like `COUNT()` and other columns whose values define groups for which you want to see results of your math functions broken out - for example, using the agency column to get `COUNT`s broken out by agency.

To break out our count of grants by agency in 2010, for example, we start with our previous query:

    SELECT COUNT( DISTINCT( * ) ) AS grant_count
    FROM projects
    WHERE ( fy = 2010 );

First we add the "agency" column to the "SELECT CLAUSE"
    
    SELECT COUNT( DISTINCT( * ) ) AS grant_count, agency
    FROM projects
    WHERE ( fy = 2010 );

Then, we also add the sex column to the `GROUP BY` clause, so it will `COUNT` the unique SSNs for each value of sex, rather than lumping both genders together:

    SELECT COUNT( DISTINCT( * ) ) AS grant_count, agency
    FROM projects
    WHERE ( fy = 2010 );
    GROUP BY agency;

## Aggregation functions

In addition to `COUNT()`, there are a number of other useful aggregation functions in SQL.  To name a few:

- **_SUM( column )_** : Calculate the sum of column for all the rows in each group
- **_AVG( column )_** : Calculate the numeric average for all of the rows in each group
- **_COUNT( column )_** : Count the number of rows in each group
- **_MIN( column ) and MAX( column )_** : Find the minimum or maximum value of column in all the rows in each group

Note a few characteristics of these aggregate functions:

- the calculation operates on an column;
- the calculation will consider every value of that column;
- the calcuation aggregates the values of the column in certain way, and return a single value instead;

## ORDER BY

When an SQL query is run, the results are not guaranteed to return in any set order, though basic single-table queries oftern return rows in the order they appear in the database. 

If you want your results ordered a certain way, you use an `ORDER BY` clause to tell the database how to order the rows in the results of a given query.

    SELECT COUNT( * ) as grant_count, agency
    FROM projects
    GROUP BY agency
    ORDER BY grant_count DESC;

In an `ORDER BY` clause one can specify a list of the columns you want to sort the results on, in the order they appear in the list.  The database will first order the rows based on the values in the left-most item in the `ORDER BY` list.  Then as it moves left-to-right through the `ORDER BY` list, when there are duplicates in a given column, if there is another column name in the list to the right of the current column, it will order each set of rows with duplicate values based on the next column named in the `ORDER BY` list.

By default, rows are ordered in ASCending order.  After you specify a given column to `ORDER BY`, you can optionally specify either ASC for ascending order, or DESC for descending order.

## JOIN: Connecting multiple tables

- Back to the [Table of Contents](#Table-of-Contents)

SQL Lets you join multiple tables together inside a single query by specifying JOIN criteria that tell the database when a row from one of the two tables can be considered a match for a row in the other.  Generally, you'll join on shared IDs (usually a ForeignKey in one table that references the ID of a row in the other), or on shared identifying information like SSN or name.

The most basic join is an INNER join, which only returns records from either of two tables you join that match your JOIN criteria.

    SELECT *
    FROM projects prj
    INNER JOIN abstracts abs
    ON prj.project_number = abs.project_number
    LIMIT 10;

You can also do an **_OUTER JOIN_** if you want records from one or both of the tables that do not match to still be included in the results.

If you want all the records from the table in the WHERE clause maintained, for example, you'd use a **_LEFT OUTER JOIN_** (for example, if you were building a data set for analysis from the abstract table, and you want to add project information for those who have it, but keep abstracts without it). Vice versa it would be a **_RIGHT OUTER JOIN_** 

A **_FULL OUTER JOIN** returns all records from each table along with all matches from the other table.  In cases where a row from either table does not have a match, the columns from the other table will be set to NULL (_be aware that with large tables, if ther eare multiple matches per row, this can create a very large result set - see "cartesian product"_).